In [1]:
from tensorflow.keras.utils import to_categorical
import jieba
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from gensim.models import Word2Vec
import json
from tensorflow.keras.layers import (Bidirectional,
                                     Embedding,
                                     GRU, 
                                     GlobalAveragePooling1D,
                                     GlobalMaxPooling1D,
                                     Concatenate,
                                     SpatialDropout1D,
                                     BatchNormalization,
                                     Dropout,
                                     Dense,
                                     Activation,
                                     concatenate,
                                     Input
                                    )                        

In [2]:
df_train = pd.read_csv('./data/nCoV_100k_train.labled.csv',engine ='python',nrows=99900)
df_test = pd.read_csv('./data/nCov_10k_test.csv',engine ='python')

In [3]:
df_train.shape

(99900, 7)

In [4]:
def cut_words(sentence):
    #print sentence
    return " ".join(jieba.cut(sentence))

In [5]:
MAX_SEQUENCE_LENGTH = 100
input_categories = '微博中文内容'
output_categories = '情感倾向'
df_train = df_train[df_train[output_categories].isin(['-1','0','1'])]

In [6]:
df_train['sentence_cuted'] =  df_train[input_categories].astype(str).apply(cut_words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\37473\AppData\Local\Temp\jieba.cache
Loading model cost 0.713 seconds.
Prefix dict has been built successfully.


In [7]:
df_test['sentence_cuted'] =  df_test[input_categories].astype(str).apply(cut_words)

In [ ]:
df_train.head()

In [13]:
df_test.head()

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频,情感倾向,sentence_cuted
0,4456072029125500,01月01日 23:50,存曦1988,写在年末冬初孩子流感的第五天，我们仍然没有忘记热情拥抱这2020年的第一天。带着一丝迷信，早...,['https://ww2.sinaimg.cn/orj360/005VnA1zly1gah...,[],0,写 在 年末 冬初 孩子 流感 的 第五天 ， 我们 仍然 没有 忘记 热情 拥抱 这 20...
1,4456074167480980,01月01日 23:58,LunaKrys,开年大模型…累到以为自己发烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#?,[],[],-1,开年 大 模型 … 累到 以为 自己 发烧 了 腰疼 膝盖 疼 腿疼 胳膊 疼 脖子 疼 #...
2,4456054253264520,01月01日 22:39,小王爷学辩论o_O,�偳癯空饩褪俏业�，爹，发烧快好，毕竟美好的假期拿来养病不太好，假期还是要好好享受快乐，爹，...,['https://ww2.sinaimg.cn/thumb150/006ymYXKgy1g...,[],1,� 偳 癯 空 饩 褪 俏 业 � ， 爹 ， 发烧 快 好 ， 毕竟 美好 的 假期 拿来...
3,4456061509126470,01月01日 23:08,芩鎟,新年的第一天感冒又发烧的也太衰了但是我要想着明天一定会好的?,['https://ww2.sinaimg.cn/orj360/005FL9LZgy1gah...,[],1,新年 的 第一天 感冒 又 发烧 的 也 太衰 了 但是 我要 想着 明天 一定 会 好 的 ?
4,4455979322528190,01月01日 17:42,changlwj,问：我们意念里有坏的想法了，天神就会给记下来，那如果有好的想法也会被记下来吗？答：那当然了。...,[],[],1,问 ： 我们 意念 里 有 坏 的 想法 了 ， 天神 就 会 给 记下来 ， 那 如果 有...


In [8]:
df_test.head()

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频,sentence_cuted
0,4456068992182160,01月01日 23:38,-精緻的豬豬女戰士-,#你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...,['https://ww2.sinaimg.cn/thumb150/745aa591ly1g...,[],# 你好 2020 # 新年 第一天 元气 满满的 早起 出门 买 早饭 结果 高估 了 自...
1,4456424178427250,01月02日 23:09,liujunyi88,大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...,[],[],大宝 又 感冒 鼻塞 咳嗽 了 ， 还有 发烧 。 队友 加班 几天 不回 。 感觉 自己 ...
2,4456797466940200,01月03日 23:53,ablsa,还要去输两天液，这天也太容易感冒发烧了，一定要多喝热水啊?,['https://ww3.sinaimg.cn/orj360/006fTidCly1gaj...,[],还要 去 输 两天 液 ， 这天 也 太 容易 感冒 发烧 了 ， 一定 要 多 喝 热水 啊 ?
3,4456791021108920,01月03日 23:27,喵吃鱼干Lynn,我太难了别人怎么发烧都没事就我一检查甲型流感?,[],[],我太难 了 别人 怎么 发烧 都 没事 就 我 一 检查 甲型 流感 ?
4,4457086404997440,01月04日 19:01,我的发小今年必脱单,果然是要病一场的喽回来第三天开始感冒今儿还发烧了喉咙眼睛都难受的一匹怎么样能不经意让我的毕设...,[],[],果然 是 要 病 一场 的 喽 回来 第三天 开始 感冒 今儿 还 发烧 了 喉咙 眼睛 都...


In [9]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns].astype(int) + 1)
outputs = compute_output_arrays(df_train, output_categories)

In [10]:
outputs

array([1, 0, 2, ..., 2, 1, 1])

In [11]:
train = df_train
test = df_test

In [12]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000, 
                                                  lower=False,filters="")# 对数据构造词典
tokenizer.fit_on_texts(train['sentence_cuted'].tolist()+test['sentence_cuted'].tolist())

json_str = tokenizer.to_json()
json.dump(json_str,open(r'tokenizer_config.json','w'))#文件路径前面加上 r 是为了避免转义

In [ ]:
train_ = tokenizer.texts_to_sequences(train['sentence_cuted'].values)# 文本 ---> 数字
test_ = tokenizer.texts_to_sequences(test['sentence_cuted'].values)# 同上

In [ ]:
maxlen = int(np.percentile(list(map(lambda x: len(x),train_)),95))# 统计95% 字的长度

In [ ]:
maxlen = 100

In [ ]:
train_ = tf.keras.preprocessing.sequence.pad_sequences(train_, maxlen=maxlen,
                                                      padding='pre',truncating='pre',value=0.0)
test_ = tf.keras.preprocessing.sequence.pad_sequences(test_, maxlen=maxlen,
                                                     padding='pre',truncating='pre',value=0.0)

In [ ]:
word_vocab = tokenizer.word_index
print(len(word_vocab))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_,outputs,
                                               test_size=0.1,random_state=666)# 0.1 或 0.05 ,因为数据量比较大，

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(64)# 打乱,随机取batch训练
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)# 测试集批处理的大小可以和训练机匹配的大小不一样?

In [ ]:
from gensim.models import Word2Vec

In [ ]:
all_data=pd.concat([train['sentence_cuted'],test['sentence_cuted']])
file_name = './word2vec/Word2Vec_word_200.model'
if not os.path.exists(file_name):
    print('training')
    model_word2vec = Word2Vec([document.split(' ')for document in all_data.values],
                     size=200, 
                     window=5,
                     iter=10, 
                     workers=11, 
                     seed=2018, 
                     min_count=2)

    model_word2vec.save(file_name)
else:
    print('loading')
    model_word2vec = Word2Vec.load(file_name)
print("add word2vec finished....")    

In [ ]:
model_word2vec.wv.vocab.keys()

In [ ]:
model_word2vec.most_similar("上海",topn=20)

In [ ]:
count = 0
word_vocal_reverse = {}
embedding_matrix = np.zeros((len(word_vocab) + 1, 200)) # 为什么加1 ? 因为字典没有0, 我们为补全造了一个0
for word, i in word_vocab.items():
    word_vocal_reverse[i]=word
    embedding_vector = model_word2vec.wv[word] if word in model_word2vec else None # 取出向量
    if embedding_vector is not None:
        count += 1
        embedding_matrix[i] = embedding_vector# i 表示词典中的位置, 通过embedding,来代替 字的向量表示
    else:
        unk_vec = np.random.random(200) * 0.5
        unk_vec = unk_vec - unk_vec.mean()# 减去均值
        embedding_matrix[i] = unk_vec

In [ ]:
np.save('idsMatrix', embedding_matrix)

word_vocal_reverse

In [26]:
def build_model(sent_length, embeddings_weight,class_num):
    content = Input(shape=(sent_length,), dtype='int32')
    embedding = Embedding(
        name="sentence_cuted",
        input_dim=embeddings_weight.shape[0],
        weights=[embeddings_weight],
        output_dim=embeddings_weight.shape[1],
        trainable=False)

    x = SpatialDropout1D(0.2)(embedding(content))

    x = Bidirectional(GRU(200, return_sequences=True))(x)
#     x = Bidirectional(GRU(200, return_sequences=True))(x)

    avg_pool = GlobalAveragePooling1D()(x)# 全军平均池化
    max_pool = GlobalMaxPooling1D()(x)# 全局最大池化

    conc = concatenate([avg_pool, max_pool]) # 特征放到一起，
    
    x = Dense(1000)(conc)# 全连接层
    x = BatchNormalization()(x)
    x = Activation(activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(500)(x)
    x = BatchNormalization()(x)
    x = Activation(activation="relu")(x)
    output = Dense(class_num, activation="softmax")(x)

    model = tf.keras.models.Model(inputs=content, outputs=output)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [27]:
model = build_model(maxlen, embedding_matrix,3)

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
sentence_cuted (Embedding)      (None, 100, 200)     32267800    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 200)     0           sentence_cuted[0][0]             
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 100, 400)     482400      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [29]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard
# 检查点保存至的目录
checkpoint_dir = './checkpoints_predict'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
print(checkpoint_dir)
print(checkpoint_prefix)

./checkpoints_predict
./checkpoints_predict\ckpt_{epoch}


In [30]:
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((89831, 100), (9982, 100), (89831,), (9982,))

In [31]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=6)
plateau = ReduceLROnPlateau(monitor="val_accuracy", 
                            verbose=1, 
                            mode='max', 
                            factor=0.5, 
                            patience=3)
checkpoint = ModelCheckpoint(checkpoint_prefix, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max',
                             save_weights_only=True)

model.fit(train_ds,
          epochs=15,
          validation_data=val_ds,
          callbacks=[early_stopping, plateau, checkpoint],
          verbose=1)

Train for 1404 steps, validate for 156 steps
Epoch 1/15
1403/1404 [============================>.] - ETA: 0s - loss: 3.7132 - accuracy: 0.8984
Epoch 00001: val_accuracy improved from -inf to 0.64606, saving model to ./checkpoints_predict\ckpt_1
1404/1404 [==============================] - 621s 442ms/step - loss: 3.7128 - accuracy: 0.8983 - val_loss: 3.6853 - val_accuracy: 0.6461
Epoch 2/15
1403/1404 [============================>.] - ETA: 0s - loss: 3.7138 - accuracy: 0.7137
Epoch 00002: val_accuracy improved from 0.64606 to 0.70276, saving model to ./checkpoints_predict\ckpt_2
1404/1404 [==============================] - 631s 450ms/step - loss: 3.7141 - accuracy: 0.7137 - val_loss: 3.7325 - val_accuracy: 0.7028
Epoch 3/15
1403/1404 [============================>.] - ETA: 0s - loss: 3.7129 - accuracy: 0.7342
Epoch 00003: val_accuracy improved from 0.70276 to 0.74664, saving model to ./checkpoints_predict\ckpt_3
1404/1404 [==============================] - 616s 439ms/step - loss: 3.7130

In [32]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [33]:
valid_prob = model.predict(val_ds)
valid_pred = np.argmax(valid_prob,axis=1)


In [34]:
y_val

array([1, 1, 1, ..., 2, 2, 1])

In [35]:


# val_true = np.argmax(y_val,axis=1)

from sklearn.metrics import f1_score
print ("valid's macro_f1: %s" % f1_score(y_val,valid_pred,average='macro'))

valid's macro_f1: 0.05142776062246388


In [36]:
df_sub = df_test[['微博id']]

In [37]:
df_sub.head()

,微博id
0,4456068992182160
1,4456424178427250
2,4456797466940200
3,4456791021108920
4,4457086404997440


In [38]:
test_ds = tf.data.Dataset.from_tensor_slices((test_,np.zeros((test_.shape[0],3)))).batch(128)
test_prob = model.predict(test_ds)

In [39]:
test_preb = np.argmax(test_prob,axis=1)

In [40]:
test_preb


array([0, 0, 2, ..., 0, 0, 0], dtype=int64)

In [41]:
df_sub['y'] = test_preb-1

c:\tempbeforenewdisk\anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
df_sub.head()

,微博id,y
0,4456068992182160,-1
1,4456424178427250,-1
2,4456797466940200,1
3,4456791021108920,1
4,4457086404997440,1


In [43]:
df_sub.columns=['id','y']

In [44]:
df_sub.head()

,id,y
0,4456068992182160,-1
1,4456424178427250,-1
2,4456797466940200,1
3,4456791021108920,1
4,4457086404997440,1


In [45]:
df_sub.to_csv('test_sub.csv',index=False, encoding='utf-8')